# Notebook handeling

In [18]:
#| default_exp notebook

## Notebook object

The underlying file structure of a notebook is JSON, which directly matches python dictionaries.  
As such, it is treated by `nbformat` as spcieal kind of `dict`.  

In [25]:
#| export

from nbformat import NotebookNode
from nbformat import read as read_nb, write as write_nb, validate as validate_nb, from_dict
import jq

Due to that, all of `jq`s' power is available to us: 

In [26]:
#| export

class Notebook:
    def __init__(self, path):
        self.path = path
        self.nb = read_nb(path, as_version=4)

    def apply_jq_pattern(self, pattern):
        return from_dict(jq.compile(pattern).input(self.nb).first())

    def clean_execution_counts(self):
        self.nb = self.apply_jq_pattern('.cells[].execution_count = null')
        
    def clean_outputs(self):
        self.nb = self.apply_jq_pattern('.cells[].outputs = []')
    
    def clean_all_metadata(self):
        self.nb = self.apply_jq_pattern('.cells[].metadata = {}')
        self.nb = self.apply_jq_pattern('.metadata = {}')
        
    def clean_all(self):
        self.clean_execution_counts()
        self.clean_outputs()
        self.clean_all_metadata()
        return self
        
    def save(self, path=None):
        path = path if path else self.path
        write_nb(self.nb, path)
        

### Tests

#### Base Clean Notebook

In [37]:
#| hide

# tests the "clean" nb is indeed clean
nb = read_nb('../tests/data/clean.ipynb', as_version=4)
validate_nb(nb)

# contains a singel cell 
assert nb.cells == [{'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ''}]

#### Cleaning a clean notebook produces same result

In [42]:
#| hide
assert nb == Notebook('../tests/data/clean.ipynb').clean_all().nb

## CLI execution

In [43]:
#| export

from fastcore.script import call_parse, Param
from pathlib import Path


@call_parse
def nbcleanse():
    "Clean all notebooks in the current directory"
    for path in Path('.').glob('*.ipynb'):
        Notebook(path).clean_all().save()

## export

In [44]:
from nbdev.export import nb_export

nb_export('nb_definition.ipynb')